In [1]:
import sys,os,copy

import numpy as np
import pickle

from time import time
from spharm import Spharmt, getspecindx, regrid
from netCDF4 import Dataset
from scipy import stats

sys.path.insert(1,'/home/disk/kalman2/mkb22/LMR_lite/')
import LMR_lite_utils as LMRlite
import LMR_utils as lmr
import LMR_config

sys.path.insert(1,'/home/disk/p/mkb22/Documents/si_utils_kb/')
import Sice_utils as siutils 

sys.path.insert(1,'/home/disk/p/mkb22/Documents/si_analysis_kb/common_era_experiments/')
import commonera_utils as ceutils 

Loading information from datasets.yml
Loading information from grid_def.yml


In [2]:
inflate = 1.0
inf_name = '1'

#prior_name = 'mpi'
prior_name = 'ccsm4'

pp_err = 0.1
#pp_err = False
serr = '0_1'
#serr = 'pages2kv2'

cfile = '/home/disk/kalman2/mkb22/LMR_lite/configs/config_ccsm4_brennan2020.yml'

proxies = 'pseudo'

savedir = ('/home/disk/p/mkb22/nobackup/LMR_output/common_era_experiments/experiments/pseudo/')

print('loading configuration...')
cfg = LMRlite.load_config_simple(cfile)

recon_start = str(cfg.core.recon_period[0])
recon_end = str(cfg.core.recon_period[1])
loc_list = [cfg.core.loc_rad]
    
iter_range = cfg.wrapper.iter_range
MCiters = range(iter_range[0],iter_range[1]+1)

proxy_ind = np.zeros((iter_range[1]+1,405))
prior_ind = np.zeros((iter_range[1]+1,200))

loading configuration...
Checking configuration ... 
OK!


In [3]:
# load proxies
prox_manager = LMRlite.load_proxies(cfg)

                                 Bivalve_d18O :     1
               Corals and Sclerosponges_Rates :     1
                Corals and Sclerosponges_SrCa :     8
                Corals and Sclerosponges_d18O :    17
                        Ice Cores_MeltFeature :     1
                               Ice Cores_d18O :    22
                                 Ice Cores_dD :     4
                              Lake Cores_Misc :     2
                             Lake Cores_Varve :     3
                       Tree Rings_WidthPages2 :   259
                       Tree Rings_WoodDensity :    47
                                        TOTAL :   365
-----------------------------------------------------
completed in 3.8840298652648926 seconds
-----------------------------------------------------


In [4]:
# All proxies available given config setup: 
type_site_assim = prox_manager.all_ids_by_group
assim_proxy_count = len(prox_manager.all_proxies)
for pkey, plist in sorted(type_site_assim.items()):
    print(('%45s : %5d' % (pkey, len(plist))))
print(('%45s : %5d' % ('TOTAL', assim_proxy_count)))

                                 Bivalve_d18O :     1
               Corals and Sclerosponges_Rates :     2
                Corals and Sclerosponges_SrCa :    11
                Corals and Sclerosponges_d18O :    28
                        Ice Cores_MeltFeature :     1
                               Ice Cores_d18O :    28
                                 Ice Cores_dD :     7
                              Lake Cores_Misc :     2
                             Lake Cores_Varve :     5
                       Tree Rings_WidthPages2 :   343
                       Tree Rings_WoodDensity :    59
                                        TOTAL :   487


In [9]:
dir(prox_manager)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'all_ids_by_group',
 'all_proxies',
 'assim_ids_by_group',
 'ind_assim',
 'ind_eval',
 'info_intervals',
 'proxy_obj_generator',
 'sites_assim_proxy_objs',
 'sites_eval_proxy_objs']

In [22]:
prox_lat =[]
prox_lon =[]
for proxy_idx, Y in enumerate(prox_manager.all_proxies):
    prox_lat = np.append(prox_lat,Y.lat)
    prox_lon = np.append(prox_lat,Y.lon)

In [36]:
prox_assim_info = {'Tree Rings_WidthPages2':{'lat':[],'lon':[],'color':'g', 'label':'Tree', 'marker':'D'},
                   'Tree Rings_WidthBreit':{'lat':[],'lon':[],'color':'g', 'label':'Tree','marker':'D'},
                   'Tree Rings_WoodDensity':{'lat':[],'lon':[],'color':'g', 'label':'Tree','marker':'D'}, 
                   'Corals and Sclerosponges_d18O':{'lat':[],'lon':[],'color':'coral', 'label':'Coral'}, 
                   'Corals and Sclerosponges_SrCa':{'lat':[],'lon':[],'color':'coral', 'label':'Coral'}, 
                   'Corals and Sclerosponges_Rates':{'lat':[],'lon':[],'color':'coral', 'label':'Coral'},
                   'Ice Cores_d18O':{'lat':[],'lon':[],'color':'darkturquoise', 'label':'Ice Core','marker':'o'},
                   'Ice Cores_dD':{'lat':[],'lon':[],'color':'darkturquoise', 'label':'Ice Core','marker':'o'}, 
                   'Ice Cores_MeltFeature':{'lat':[],'lon':[],'color':'darkturquoise', 'label':'Ice Core','marker':'o'},
                   'Ice Cores_Accumulation':{'lat':[],'lon':[],'color':'darkturquoise', 'label':'Ice Core','marker':'o'},
                   'Lake Cores_Varve':{'lat':[],'lon':[],'color':'royalblue', 'label':'Lake Core','marker':'^'}, 
                   'Lake Cores_Misc':{'lat':[],'lon':[],'color':'royalblue', 'label':'Lake Core','marker':'^'}, 
                   'Bivalve_d18O':{'lat':[],'lon':[],'color':'saddlebrown', 'label':'Bivalve'}}

for proxy_idx, Y in enumerate(prox_manager.all_proxies):
    prox_assim_info[Y.type]['lat'] = np.append(prox_assim_info[Y.type]['lat'],Y.lat)
    prox_assim_info[Y.type]['lon'] = np.append(prox_assim_info[Y.type]['lon'],Y.lon)

In [37]:
savedir = ('/home/disk/p/mkb22/nobackup/LMR_output/common_era_experiments/experiments/pseudo/')
savename = 'pages2kv2_all_proxy_info.pkl'

In [38]:
print('Saving experiment to: ',savedir+savename)
pickle.dump(prox_assim_info,open(savedir+savename, "wb"))

Saving experiment to:  /home/disk/p/mkb22/nobackup/LMR_output/common_era_experiments/experiments/pseudo/pages2kv2_all_proxy_info.pkl
